In [ ]:
%%bash
lein uberjar

In [ ]:
%classpath add jar ../target/jvm/uberjar/podsm-0.1.0-SNAPSHOT-standalone.jar
(clojure.lang.Compiler/loadFile "jupyter_helpers.clj")
(require '[clojure.java.io :as io]
         '[clojure.string :as string]
         '[clojure.pprint :refer [pprint print-table]]
         '[jupyter-helpers :refer [run-and-save-experiments load-experiment site-summary-table experiment-timeline experiments-summary-table
                                   display-html confusion-summary display-experiment-models get-best-experiment-label load-experiment-pair
                                   get-timing-evaluation timing-table get-experiment-block-accuracies experiment-block-accuracies-table
                                   save-data load-data]]
         '[podsm.classifier.distributed.sites :refer [make-site-structure p-site t-site]]
         '[podsm.evaluation :refer [get-order-summary]]
         '[podsm.classifier.moa-classifier :refer [adaptive-random-forest naive-bayes]]
         '[podsm.dataset.csv-dataset :refer [read-csv-dataset]]
         '[podsm.utils.stats :refer [mann-whitney-u-test wilcoxon-signed-rank-test]])

## Experiment Setup

In [ ]:
;; Dataset Configuration

(defn dataset-fn []
    (read-csv-dataset "datasets/gas/gas.csv"
                      999999 ;; All records
                      ;; Numeric features (all excluding class)
                      (map #(str "f" %) (range 1 129))))
(def dataset-name "gas")
(def feature-count 128) ;; Not including class
(def features-per-site 8)
(def trouble-factors (map #(* features-per-site %) [1 1.5 2]))
(def p-sites
    [(p-site :sensor-1 (range 0 8))
     (p-site :sensor-2 (range 8 16))
     (p-site :sensor-3 (range 16 24))
     (p-site :sensor-4 (range 24 32))
     (p-site :sensor-5 (range 32 40))
     (p-site :sensor-6 (range 40 48))
     (p-site :sensor-7 (range 48 56))
     (p-site :sensor-8 (range 56 64))
     (p-site :sensor-9 (range 64 72))
     (p-site :sensor-10 (range 72 80))
     (p-site :sensor-11 (range 80 88))
     (p-site :sensor-12 (range 88 96))
     (p-site :sensor-13 (range 96 104))
     (p-site :sensor-14 (range 104 112))
     (p-site :sensor-15 (range 112 120))
     (p-site :sensor-16 (range 120 128))])

(def dataset (dataset-fn))
(defn dataset-fn [] dataset)

In [ ]:
;; Experiment Configuration

(def base-classifier naive-bayes)
(def trouble-classifier adaptive-random-forest)

(def base-setup
    {:dataset-description (keyword dataset-name)
     :dataset-fn dataset-fn
     :trouble-classifier trouble-classifier
     :base-site-structure (apply make-site-structure feature-count p-sites)
     ;; Disable monitor logging because this is a long-running experiment
     ;; where detailed logging data is excessively large.
     :disable-monitor-logging true
     :batch1 5000
     })

(def aggregation-rules {:max-conf {:type :max-conf}
                        :simple-voting {:type :simple-voting}
                        :stacked {:type :two-level-stacked
                         :classifier-generator trouble-classifier}
                        
                        })
                       

(def base-system-config
    {:site-window-size 1000
     :site-training-time 0
     :shared-sources? false
     :creation-window-size 1000
     :creation-time-threshold 500
     :removal-window-size 1000
     :removal-time-threshold 500
     :creation-agreement-threshold {:type :smoothed-hoeffding-bound
                                    :r 1 :delta 0.001 :sharpness 5}
     :removal-accuracy-threshold {:type :hoeffding-bound
                                  :r 1 :delta 0.001}
     :removal-usage-threshold 0.05})



(def podsm-experiments
    (for [trouble-factor trouble-factors
          [aggregation-rule-key aggregation-rule] aggregation-rules]
        (assoc base-setup
            :label (keyword (str dataset-name "-podsm"
                                 aggregation-rule-key
                                 "-tf" trouble-factor))
            :p-site-aggregation-rule aggregation-rule
            :system-config (assoc base-system-config
                               :trouble-factor trouble-factor))))

(def experiments podsm-experiments)

(def output-dir (str "workspace/" dataset-name))

## Run Experiments

In [ ]:
(import '[com.twosigma.beakerx.widget Output])
(def experiment-output (Output.))
experiment-output

In [ ]:
(run-and-save-experiments output-dir base-classifier experiments
                          :beaker-output experiment-output
                          :thread-count 1)

## Experiments Summary

In [ ]:
(def experiments-summary
    (load-experiment output-dir :summary))

(experiments-summary-table experiments-summary)

## Accuracy and Transmission Over Time 

In [ ]:
(def performance-measure [:skip-1000 :accuracy])
(def partition-size 100)

### Maximum Confidence Aggregation

In [ ]:
(def max-conf-experiments (load-experiment-pair output-dir experiments-summary performance-measure :max-conf))

In [ ]:
(let [experiments max-conf-experiments
      result-count (count (:results (:podsm experiments)))]
    (.display (experiment-timeline (:podsm experiments)
                                   (dissoc experiments :podsm)
                                   :partition-size partition-size
                                   :x-bounds [0 result-count]
                                   :y-bounds-acc [0 1.2]
                                   :y-bounds-data [0 1.5]
                                   :event-rows 6)))

### Simple Voting Aggregation

In [ ]:
(def voting-experiments (load-experiment-pair output-dir experiments-summary performance-measure :simple-voting))

In [ ]:
(let [experiments voting-experiments
      result-count (count (:results (:podsm experiments)))]
    (.display (experiment-timeline (:podsm experiments)
                                   (dissoc experiments :podsm)
                                   :partition-size partition-size
                                   :x-bounds [0 result-count]
                                   :y-bounds-acc [0 1.5]
                                   :y-bounds-data [0 1.5]
                                   :event-rows 6)))

### Stacked Aggregation

In [ ]:
(def stacked-experiments (load-experiment-pair output-dir experiments-summary performance-measure :stacked))

In [ ]:
(let [experiments stacked-experiments
      result-count (count (:results (:podsm experiments)))]
    (.display (experiment-timeline (:podsm experiments)
                                   (dissoc experiments :podsm)
                                   :partition-size partition-size
                                   :x-bounds [0 result-count]
                                   :y-bounds-acc [0.5 1.5]
                                   :y-bounds-data [0 0.7]
                                   :event-rows 6)))

## Timing Evaluation

In [ ]:
(def timing-evaluation (get-timing-evaluation output-dir
                                              (distinct (map :label experiments-summary)) ;; Drop repeated experiment labels
                                              1000))

In [ ]:
(save-data (str output-dir "/:timing-summary.edn") timing-evaluation)

In [ ]:
(timing-table timing-evaluation)

## Usage and Accuracy of Site Order Tiers

In [ ]:
(let [experiment (load-experiment output-dir :gas-podsm:simple-voting-tf16)]
    (for [[order {:keys [usage accuracy unweighted-f mean-critical-path-time mean-time-between-responses]}]
      (get-order-summary (drop 1000 (:results experiment)) (:final-site-structure experiment))]
        {"Order" order
         "Usage" usage
         "Accuracy" accuracy
         "Unweighted Mean F-Score" unweighted-f
         "Mean Critical Path Time" mean-critical-path-time
         "Mean Time Between Responses" mean-time-between-responses}))

## Centralised Accuracy

In [ ]:
(def central-output-dir (str output-dir "/central"))

In [ ]:
(def centralise-all-trouble-factor (* 2 feature-count))

(def centralised-experiment
    (assoc base-setup
        :label (keyword (str dataset-name "-centralised"))
        :base-site-structure
        (apply make-site-structure feature-count
            (concat p-sites [(t-site :central (map :label p-sites) centralise-all-trouble-factor)]))
        :p-site-aggregation-rule {:type :max-conf}
        :disable-monitors true
        :system-config (assoc base-system-config
                               :trouble-factor centralise-all-trouble-factor
                               :creation-agreement-threshold 999
                               :removal-accuracy-threshold -999
                               :removal-usage-threshold -999)))

(run-and-save-experiments central-output-dir trouble-classifier [centralised-experiment]
                          :thread-count 1)

In [ ]:
(def central-experiments-summary
    (load-experiment central-output-dir :summary))
    
(experiments-summary-table central-experiments-summary)